<a href="https://colab.research.google.com/github/mostly-sunny/digital-health-hackathon/blob/main/3.%20coxph_find_best_network_by_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pycox - CoxPH Model
- Network : Test with sets, and find best network and lr by lowest brier score
- Input Variables : G1 ~ G300, Var1 ~ Var10, Treatment
- Output Variables : time, event
- Scaler : MinMaxScaler -> Var1 ~ Var10



In [110]:
pip install pycox

In [111]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper
import pandas as pd

import torch
import torchvision
import torchtuples as tt

from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [112]:
torch.cuda.is_available()

True

In [113]:
np.random.seed(123456)
_ = torch.manual_seed(123456)

- all-in-one.csv 파일은 유전자 변이 유무, 임상 변수, 생존 기간, 사망 여부, 치료 유무가 열로 존재하는 파일
- test-data-treat-and-untreat.csv 파일은 all-in-one의 열은 같은 602개의 데이터.
  - (0번째 행) : 유전자 변이 모두 0, 치료 0
  - (1번째 행) : 유전자 변이 모두 0, 치료 1
  - (2~301번째 행) : 유전자 변이 n-1에만 1, 치료 0
  - (302~601번재 행) : 유전자 변이 n-301에만 1, 치료 1
- pandas 라이브러리에 있는 csv 파일을 DataFrame으로 바꾸어주는 read_csv 함수를 이용하여 파일을 읽어 들임.
- DataFrame은 표를 나타내는 데이터 타입임.

In [114]:
dataset = pd.read_csv('/content/all-in-one.csv')
dataset_for_hr = pd.read_csv('/content/test-data-treat-and-untreat.csv')

- 위에서 읽어들인 dataset 중에서 20%는 검증(_val -> validation)을 위해 sampling 한다.
- 남은 80%의 데이터에서도 20%는 테스트(_test)를 위해 sampling 한다.

In [115]:
dataset_test = dataset.sample(frac=0.3, random_state = np.random.seed(123456))
dataset_train = dataset.drop(dataset_test.index)
dataset_val = dataset_train.sample(frac=0.2, random_state = np.random.seed(123456))
dataset_train = dataset_train.drop(dataset_val.index)

In [116]:
dataset_test

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38,G39,G40,...,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,time,event,Treatment
601,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,5,4,6,4,3,3,5,7,1,71.955766,1,1
357,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,2,2,2,4,2,4,0,2,1,39.587305,1,0
945,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,3,4,1,5,6,2,1,2,2,3,99.459796,1,1
828,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,2,1,5,3,6,6,3,3,1,2,67.195341,1,1
649,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,3,1,2,4,0,1,1,0,6,35.031129,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,4,0,3,3,1,0,3,0,9,4,66.602950,1,0
913,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,4,4,3,9,5,3,5,0,2,2,57.497019,1,1
483,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,5,3,3,1,1,2,3,2,0,3,40.966792,0,0
276,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,4,2,0,1,5,3,5,5,4,1,107.690607,1,1


- columns_standardize : 임상변수 - 0~9사이의 값을 표준화
- columns_leave : 유전자 변이 유무 + 치료 유무 - 0과 1로 표현돼 있기 때문에 표준화 필요 없음.
- DataFrameMapper는 pandas DataFrame에서 원하는 열을 뽑아서 리스트로 만들어줌.
- 리스트로 만들때 StandardScaler()가 포함된 열은 표준화를 시킨 뒤, 그리고 None이면 갖고 있는 값을 그대로 넣음.

In [117]:
columns_standardize = ['Var' + str(i) for i in range(1,11)]
columns_leave = ['G' + str(i) for i in range(1,301)]
columns_leave += ['Treatment']

# standardize = [([col], StandardScaler()) for col in columns_standardize]
standardize = [([col], MinMaxScaler()) for col in columns_standardize]

leave = [(col, None) for col in columns_leave]

x_mapper = DataFrameMapper(leave + standardize)

- 위에서 만든 DataFrameMapper로 DataFrame 중 x(입력) 데이터를 모델이 학습할 수 있게끔 리스트 형식으로 바꾸어 준다.



In [118]:
x_train = x_mapper.fit_transform(dataset_train).astype('float32')
x_val = x_mapper.transform(dataset_val).astype('float32')
x_test = x_mapper.transform(dataset_test).astype('float32')
x_for_hr = x_mapper.transform(dataset_for_hr).astype('float32')

- DataFrame (표)에서 Y(출력)데이터인 time(생존기간)과 event(사망여부)를 뽑아 출력 데이터를 추린다.
- 검증(Validation)을 위한 입력-출력 세트 val을 만든다.

In [119]:
get_target = lambda df: (df['time'].values, df['event'].values)
y_train = get_target(dataset_train)
y_val = get_target(dataset_val)

durations_test, events_test = get_target(dataset_test)
val = x_val, y_val

함수 make_net : network을 생성해 리턴하는 함수
- input과 output의 노드 수, 은닉층 수, 은닉층의 노드 수 설정 가능

In [120]:
def make_net(in_features, out_features, hidden, nodes):
  if hidden == 1:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 2:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 3:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 4:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
      
      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, out_features)
    )
  return network

- in_features : 입력데이터의 개수 (x_train.shape : 311 = 300(유전자) + 10(임상변수) + 1(치료유무))
- out_features : 출력노드의 개수

- hidden_layers : 은닉층 수를 가지고 있는 리스트
- number_nodes : 은닉층에 있는 노드 수를 가지고 있는 리스트
- learning_rates : 테스트할 학습률을 가지고 있는 리스트
- brier_scores = brier score을 계산해 append

In [121]:
in_features = x_train.shape[1]
out_features = 1

hidden_layers = [1,2,3]
number_nodes = [1024, 2048, 3072]
learning_rates = [0.0001, 0.001, 0]
brier_scores = []

total_num = len(hidden_layers) * len(number_nodes) * len(learning_rates)
count = 1

for i in hidden_layers:
  for j in number_nodes:
    for k in learning_rates:
      print(count, '/' , total_num)
      net = make_net(in_features, out_features, i, j)
      model = CoxPH(net, tt.optim.Adam)
      batch_size = 559

      if k == 0:
        lrfinder = model.lr_finder(x_train, y_train, batch_size, tolerance = 10)
        model.optimizer.set_lr(lrfinder.get_best_lr())
      else:
        model.optimizer.set_lr(k)
      
      epochs = 512
      callbacks = [tt.callbacks.EarlyStopping()]
      verbose = True

      %%time
      model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose, val_data=val, val_batch_size=batch_size)
      _ = model.compute_baseline_hazards()
      surv = model.predict_surv_df(x_test)

      count += 1

      # calculate ratio
      log_partial_hazard = model.predict(x_for_hr)
      partial_hazard = [np.exp(lph) for lph in log_partial_hazard]

      treat_hr = []
      # ratio with treated and untreated
      for g in range(300):
        treat_hr.append([partial_hazard[g+302]/partial_hazard[g+2],'G' + str(g+1)])
      treat_hr.sort()

      # evaluation
      ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
      time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
      score = ev.integrated_brier_score(time_grid)
      nbll = ev.integrated_nbll(time_grid)

      if k == 0:
        brier_scores.append([score, nbll, i, j, lrfinder.get_best_lr(), treat_hr[:10]])
      else:
        brier_scores.append([score, nbll, i, j, k, treat_hr[:10]])

1 / 27
CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.63 µs
0:	[0s / 0s],		train_loss: 5.4769,	val_loss: 3.9904
1:	[0s / 0s],		train_loss: 5.4518,	val_loss: 3.9899
2:	[0s / 0s],		train_loss: 5.3835,	val_loss: 3.9892
3:	[0s / 0s],		train_loss: 5.3527,	val_loss: 3.9885
4:	[0s / 0s],		train_loss: 5.3057,	val_loss: 3.9876
5:	[0s / 0s],		train_loss: 5.2652,	val_loss: 3.9868
6:	[0s / 0s],		train_loss: 5.2339,	val_loss: 3.9859
7:	[0s / 0s],		train_loss: 5.1944,	val_loss: 3.9849
8:	[0s / 0s],		train_loss: 5.1530,	val_loss: 3.9838
9:	[0s / 0s],		train_loss: 5.1158,	val_loss: 3.9826
10:	[0s / 0s],		train_loss: 5.0930,	val_loss: 3.9814
11:	[0s / 0s],		train_loss: 5.0615,	val_loss: 3.9801
12:	[0s / 0s],		train_loss: 5.0279,	val_loss: 3.9787
13:	[0s / 0s],		train_loss: 5.0151,	val_loss: 3.9773
14:	[0s / 0s],		train_loss: 4.9827,	val_loss: 3.9758
15:	[0s / 0s],		train_loss: 4.9587,	val_loss: 3.9742
16:	[0s / 0s],		train_loss: 4.9354,	val_loss: 3.9726
17:	[0s / 0s],		train_loss: 4.9002

- brier_score가 가장 작은 것부터 정렬


In [122]:
treat_hr

[[array([0.5576843], dtype=float32), 'G105'],
 [array([0.57857794], dtype=float32), 'G164'],
 [array([0.5787624], dtype=float32), 'G193'],
 [array([0.5796918], dtype=float32), 'G140'],
 [array([0.58168566], dtype=float32), 'G263'],
 [array([0.5876527], dtype=float32), 'G150'],
 [array([0.58892334], dtype=float32), 'G227'],
 [array([0.5902507], dtype=float32), 'G6'],
 [array([0.59037685], dtype=float32), 'G139'],
 [array([0.5917167], dtype=float32), 'G159'],
 [array([0.5920164], dtype=float32), 'G43'],
 [array([0.5921816], dtype=float32), 'G257'],
 [array([0.59253263], dtype=float32), 'G57'],
 [array([0.5936999], dtype=float32), 'G225'],
 [array([0.5938935], dtype=float32), 'G47'],
 [array([0.5945177], dtype=float32), 'G196'],
 [array([0.59464544], dtype=float32), 'G130'],
 [array([0.5953177], dtype=float32), 'G291'],
 [array([0.5961307], dtype=float32), 'G96'],
 [array([0.59664404], dtype=float32), 'G3'],
 [array([0.59797543], dtype=float32), 'G286'],
 [array([0.59810233], dtype=float3

In [123]:
brier_scores.sort()
selected_genes = []
for i in range(10):
  selected_genes.append(brier_scores[0][5][i][1])

In [124]:
gene_count= {}
for i in brier_scores:
  lst = []
  for j in i[5]:
    lst.append(j[1])
  print(lst)
  for k in lst:
    if k in gene_count.keys():
      gene_count[k] += 1
    else:
      gene_count[k] = 1
sorted(gene_count.items(), key=lambda x: x[1], reverse=True)

['G167', 'G101', 'G96', 'G253', 'G121', 'G35', 'G200', 'G133', 'G229', 'G119']
['G137', 'G35', 'G186', 'G150', 'G8', 'G39', 'G159', 'G21', 'G31', 'G280']
['G35', 'G161', 'G127', 'G221', 'G136', 'G186', 'G280', 'G230', 'G168', 'G23']
['G193', 'G266', 'G258', 'G250', 'G267', 'G219', 'G212', 'G90', 'G17', 'G85']
['G205', 'G73', 'G89', 'G286', 'G115', 'G213', 'G231', 'G72', 'G274', 'G43']
['G157', 'G194', 'G19', 'G47', 'G124', 'G258', 'G79', 'G179', 'G220', 'G10']
['G149', 'G121', 'G230', 'G7', 'G145', 'G280', 'G298', 'G60', 'G256', 'G292']
['G41', 'G76', 'G197', 'G33', 'G54', 'G198', 'G94', 'G123', 'G158', 'G9']
['G224', 'G298', 'G176', 'G180', 'G53', 'G149', 'G89', 'G263', 'G9', 'G115']
['G271', 'G97', 'G221', 'G121', 'G7', 'G203', 'G248', 'G92', 'G191', 'G33']
['G256', 'G217', 'G148', 'G88', 'G145', 'G174', 'G91', 'G238', 'G232', 'G267']
['G288', 'G92', 'G87', 'G107', 'G15', 'G28', 'G180', 'G145', 'G89', 'G72']
['G160', 'G253', 'G297', 'G237', 'G130', 'G214', 'G150', 'G48', 'G171', 'G21

[('G35', 4),
 ('G186', 4),
 ('G159', 4),
 ('G89', 4),
 ('G149', 4),
 ('G298', 4),
 ('G121', 3),
 ('G150', 3),
 ('G280', 3),
 ('G161', 3),
 ('G221', 3),
 ('G230', 3),
 ('G193', 3),
 ('G258', 3),
 ('G267', 3),
 ('G73', 3),
 ('G115', 3),
 ('G145', 3),
 ('G256', 3),
 ('G123', 3),
 ('G263', 3),
 ('G217', 3),
 ('G148', 3),
 ('G297', 3),
 ('G214', 3),
 ('G262', 3),
 ('G253', 2),
 ('G39', 2),
 ('G31', 2),
 ('G250', 2),
 ('G212', 2),
 ('G17', 2),
 ('G85', 2),
 ('G231', 2),
 ('G72', 2),
 ('G43', 2),
 ('G157', 2),
 ('G194', 2),
 ('G19', 2),
 ('G124', 2),
 ('G79', 2),
 ('G7', 2),
 ('G60', 2),
 ('G197', 2),
 ('G33', 2),
 ('G94', 2),
 ('G158', 2),
 ('G9', 2),
 ('G180', 2),
 ('G271', 2),
 ('G97', 2),
 ('G203', 2),
 ('G92', 2),
 ('G191', 2),
 ('G174', 2),
 ('G232', 2),
 ('G28', 2),
 ('G48', 2),
 ('G243', 2),
 ('G64', 2),
 ('G282', 2),
 ('G68', 2),
 ('G140', 2),
 ('G11', 2),
 ('G195', 2),
 ('G12', 2),
 ('G29', 2),
 ('G156', 2),
 ('G42', 2),
 ('G105', 2),
 ('G192', 2),
 ('G78', 2),
 ('G104', 2),
 ('G167

- brier_score가 가장 좋은 케이스 출력

In [125]:
print("Brier Score :", brier_scores[0][0])
print("Hidden Layers :", brier_scores[0][1])
print("Number of Nodes :", brier_scores[0][2])
print("Learning Rate :", brier_scores[0][3])
print("Selection :", selected_genes)

Brier Score : 0.06956388030883823
Hidden Layers : 0.3688274601490662
Number of Nodes : 3
Learning Rate : 2048
Selection : ['G167', 'G101', 'G96', 'G253', 'G121', 'G35', 'G200', 'G133', 'G229', 'G119']


NAN 값을 기준으로 정렬이 끊어진 문제 발견

In [126]:
print("brier_score, hidden layer, number of nodes, learning rate")
for i in brier_scores:
  print(i[:-1])

brier_score, hidden layer, number of nodes, learning rate
[0.06956388030883823, 0.3688274601490662, 3, 2048, 0.001]
[0.0714698149892921, 0.4575234837936272, 2, 3072, 0.001]
[0.07172977845850954, 0.31298809673952677, 1, 1024, 0.0037649358067924866]
[0.07212109598982815, 0.33189196623890993, 2, 3072, 0.0001]
[0.07213605893051295, 0.32774831300239193, 2, 3072, 0.0001]
[0.07214706433550182, 0.3104230509928474, 1, 3072, 0.001]
[0.07221139050114526, 0.3926088606596568, 2, 2048, 0.001]
[0.07259562257438919, 0.3900376900440685, 3, 1024, 0.001]
[0.0730430432501528, 0.44463231065417536, 2, 2048, 0.0001]
[0.0730506862342299, 0.29519761922890486, 3, 2048, 0.0001]
[0.07306786096990636, 0.42387721406736667, 3, 1024, 0.0010235310218990308]
[0.07320974193268515, 0.2455116521080564, 3, 3072, 0.001]
[0.0734653310453062, 0.3710725065363887, 1, 1024, 0.001]
[0.07366918359053586, 0.34071196198941794, 1, 2048, 0.001]
[0.07392347969049229, 0.3619064945177342, 1, 3072, 0.0003351602650938856]
[0.07409526767532